## Оптимизация выполнения кода, векторизация, Numba

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Материалы:
* Макрушин С.В. Лекция 3: Оптимизация выполнения кода, векторизация, Numba
* IPython Cookbook, Second Edition (2018), глава 4
* https://numba.pydata.org/numba-doc/latest/user/5minguide.html

## Задачи для совместного разбора

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [2]:
import numpy as np 
A = np.random.randint(0, 1001, (1, 1_000_000), 'int64')
B = np.array([el+100 for el in A])
print(f'A: {A.mean()}, B: {B.mean()}')

A: 500.336696, B: 600.336696


2. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [13]:
import pandas as pd
from itertools import permutations
a = pd.DataFrame(index=range(2_000_000))
a['A'] = np.random.randint(0, 1000, (2_000_000, ))
a['B'] = np.random.randint(0, 1000, (2_000_000, ))
a['C'] = np.random.randint(0, 1000, (2_000_000, ))
a['D'] = np.random.randint(0, 1000, (2_000_000, ))
a['key'] = list(permutations(list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'), 5))[:2_000_000]
for el in a.iterrows():
    if 'A' in el[1]['key'] and \
    'B' in el[1]['key'] and \
    'C' in el[1]['key'] and \
    'D' in el[1]['key'] and \
    'E' in el[1]['key']:
        print(el)

(0, A                  468
B                   28
C                  464
D                  162
key    (A, B, C, D, E)
Name: 0, dtype: object)
(22, A                  357
B                  177
C                  688
D                  568
key    (A, B, C, E, D)
Name: 22, dtype: object)
(506, A                  662
B                  676
C                  145
D                  456
key    (A, B, D, C, E)
Name: 506, dtype: object)
(528, A                  353
B                  462
C                  768
D                  508
key    (A, B, D, E, C)
Name: 528, dtype: object)
(1012, A                  679
B                  803
C                  605
D                  481
key    (A, B, E, C, D)
Name: 1012, dtype: object)
(1034, A                  896
B                  256
C                  989
D                   21
key    (A, B, E, D, C)
Name: 1034, dtype: object)
(12144, A                  237
B                  887
C                  936
D                   14
key    (A, C, B, D, 

## Лабораторная работа 3

In [35]:
!pip install line_profiler
from line_profiler import LineProfiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


1. В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

Реализуйте несколько вариантов функции подсчета среднего значения столбца `rating` из таблицы `reviews` для отзывов, оставленных в 2010 году.

A. С использованием метода `DataFrame.iterrows` исходной таблицы;

Б. С использованием метода `DataFrame.iterrows` таблицы, в которой сохранены только отзывы за 2010 год;

В. С использованием метода `Series.mean`.

Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций.


In [19]:
recipes = pd.read_csv('/content/drive/MyDrive/tod lr2/recipes_sample.csv')
reviews = pd.read_csv('/content/drive/MyDrive/tod lr2/reviews_sample.csv')
reviews['date'] = pd.to_datetime(reviews['date'])

In [37]:
%%time
def func():
    df = pd.DataFrame(data=reviews[(reviews['date'] >= '2010-01-01') & (reviews['date'] <= '2010-12-31')])
    mean = 0
    leng = 0
    for _, row in df.iterrows():
        mean += row['rating']
        leng += 1
    print(f'A) {mean/leng}')
func()

A) 4.4544402182900615
CPU times: user 856 ms, sys: 61.9 ms, total: 918 ms
Wall time: 927 ms


In [ ]:
%%time
df = pd.DataFrame(data=reviews[(reviews['date'] >= '2010-01-01') & (reviews['date'] <= '2010-12-31')])
mean = 0
leng = 0
for _, row in df.iterrows():
    mean += row['rating']
    leng += 1
print(f'Б) {mean/leng}')

Б) 4.4544402182900615
CPU times: user 545 ms, sys: 2.01 ms, total: 547 ms
Wall time: 554 ms


In [ ]:
%%time
print(f"{df['rating'].mean()}")

4.4544402182900615
CPU times: user 676 µs, sys: 0 ns, total: 676 µs
Wall time: 690 µs


2. Какая из созданных функций выполняется медленнее? Что наиболее сильно влияет на скорость выполнения? Для ответа использовать профайлер `line_profiler`. Сохраните результаты работы профайлера в отдельную текстовую ячейку и прокомментируйте результаты его работы.

(*). Сможете ли вы ускорить работу функции 1Б, отказавшись от использования метода `iterrows`, но не используя метод `mean`?

A) 4.4544402182900615
Timer unit: 1e-09 s

Total time: 2.2507 s

Could not find file <timed exec>
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     1                                           
     2         1   13367598.0 13367598.0      0.6  
     3         1        357.0    357.0      0.0  
     4         1        245.0    245.0      0.0  
     5     12094 1941211419.0 160510.3     86.2  
     6     12094  284648268.0  23536.3     12.6  
     7     12094   11305679.0    934.8      0.5  
     8         1     167343.0 167343.0      0.0  

In [ ]:
lp = LineProfiler()
lp_wrapper = lp(func)
lp_wrapper()
lp.print_stats()

In [ ]:
%%time
df = pd.DataFrame(data=reviews[(reviews['date'] >= '2010-01-01') & (reviews['date'] <= '2010-12-31')])
mean = 0
leng = 0
for el in df['rating']:
    mean += el
    leng += 1
print(f'{mean/leng}')

4.4544402182900615
CPU times: user 16.9 ms, sys: 0 ns, total: 16.9 ms
Wall time: 18.6 ms


3. Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. Измерьте время выполнения этой функции. Сможете ли вы найти узкие места в коде, используя профайлер? Выпишите (словами), что в имеющемся коде реализовано неоптимально. Оптимизируйте функцию и добейтесь значительного (как минимум, на один порядок) прироста в скорости выполнения.

Total time: 47.7266 s
Слабые места: лишние присвоения переменных, 2 перебора, использование списков

In [91]:
def get_word_reviews_count(df):
    word_reviews = {}
    for row in df['review']:
        for word in str(row).split():
            if word not in word_reviews:
                word_reviews[word] = 1
            else:
                word_reviews[word] += 1
    return word_reviews
get_word_reviews_count(reviews)

{'Last': 100,
 'week': 804,
 'whole': 5630,
 'sides': 313,
 'of': 109040,
 'frozen': 2648,
 'salmon': 729,
 'fillet': 60,
 'was': 88793,
 'on': 34590,
 'sale': 149,
 'in': 61551,
 'my': 44166,
 'local': 561,
 'supermarket,': 10,
 'so': 46106,
 'I': 288141,
 'bought': 1369,
 'tons': 161,
 '(okay,': 5,
 'only': 13967,
 '3,': 48,
 'but': 42528,
 'total': 381,
 'weight': 160,
 'over': 9066,
 '10': 2305,
 'pounds).': 3,
 'This': 39937,
 'recipe': 41128,
 'is': 55083,
 'perfect': 4400,
 'for': 121248,
 'fillet,': 14,
 'even': 7881,
 'though': 2315,
 'it': 111224,
 'calls': 520,
 'steaks.': 96,
 'cut': 6689,
 'up': 13585,
 'the': 266099,
 'into': 7035,
 'individual': 314,
 'portions': 156,
 'and': 217925,
 'followed': 4861,
 'instructions': 731,
 'exactly.': 578,
 "I'm": 7227,
 'one': 15090,
 'those': 2287,
 'food': 2416,
 'combining': 74,
 'diets,': 5,
 'left': 4691,
 'out': 23647,
 'white': 3426,
 'wine': 1258,
 'added': 21723,
 'just': 24955,
 'a': 166160,
 'dash': 532,
 'vinegar': 1273,
 

In [46]:
def get_word_reviews_count1(df):
    word_reviews = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        for word in row['review'].split(' '):
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
    
    word_reviews_count = {}
    for _, row in df.dropna(subset=['review']).iterrows():
        review = row['review']
        words = review.split(' ')
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

In [92]:
lp = LineProfiler()
lp_wrapper = lp(get_word_reviews_count)
lp_wrapper(reviews)
lp.print_stats()

Timer unit: 1e-09 s

Total time: 8.02745 s
File: <ipython-input-91-3c7f22ada245>
Function: get_word_reviews_count at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def get_word_reviews_count(df):
     2         1       1283.0   1283.0      0.0      word_reviews = {}
     3    126696   74360212.0    586.9      0.9      for row in df['review']:
     4   6589887 2153386922.0    326.8     26.8          for word in str(row).split():
     5   6425616 2678809086.0    416.9     33.4              if word not in word_reviews:
     6    164271   80713377.0    491.3      1.0                  word_reviews[word] = 1
     7                                                       else:
     8   6425616 3040174372.0    473.1     37.9                  word_reviews[word] += 1
     9         1        738.0    738.0      0.0      return word_reviews



#### [версия 2]
* Уточнены формулировки задач 1, 3, 4